In [4]:
!pip install mysql-connector-python
!pip install pyngrok
!pip install pymysql
!pip install sqlalchemy
!pip install boto3




In [5]:
!ngrok authtoken "***"





Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [29]:
from sqlalchemy import create_engine, text
from pyngrok import ngrok
import pandas as pd
from datetime import date
import os
from datetime import datetime, timedelta
import boto3
#создаем подкючение к бд
ngrok_host = '***'
ngrok_port = ****
username = 'root'
password = 'root'
database = 'mysqlproject_2'

connection_string = f"mysql+pymysql://{username}:{password}@{ngrok_host}:{ngrok_port}/{database}"


engine = create_engine(connection_string)


with engine.connect() as connection:
      connection.execute(text("SET character_set_results = 'latin1';"))
      connection.execute(text("SET character_set_client = 'latin1';"))
      connection.execute(text("SET character_set_connection = 'latin1';"))


In [7]:
#делаем курсоры
def sql(query):
    df = pd.read_sql(query, engine)
    df.index = df.index + 1
    return df

def execute(query):
  with engine.connect() as connection:
    connection.execute(text(query))
    connection.commit()


In [8]:
#просто смотрим на самую информативную и полнуб таблицу бд
query = '''
select
        o.order_id
      , o.order_date
      , c.name cus_name
      , address
      , service_name
      , price
      , co.name cour_name
      , co.phone
      , transport_name
      , z.zone_name
from
        Orders o
inner join
        Customers c on c.customer_id = o.customer_id
inner join Services s
        on s.service_id = o.service_id
inner join
        Couriers co on co.courier_id = o.courier_id
inner join
        Transport_Types tt on tt.transport_id = co.transport_type
inner join
        Zones z on z.zone_id = co.zone
order by
        order_id;
'''
sql(query)

,order_id,order_date,cus_name,address,service_name,price,cour_name,phone,transport_name,zone_name
1,1,2025-03-19 15:00:00,Сергей,"ул. Пушкина, 15",Курьерская доставка,700.0,Мария,79990004444,Велосипед,ЦАО
2,2,2025-03-17 11:00:00,Ольга,"ул. Гагарина, 10",Курьерская доставка,700.0,Дмитрий,79990002222,Велосипед,ЮВАО
3,3,2025-03-17 12:00:00,Сергей,"ул. Пушкина, 15",Экспресс-доставка,1200.0,АНтон,79990003333,Мотоцикл,ЗАО
4,4,2025-03-17 13:00:00,Анна,"ул. Мира, 20",Доставка еды,500.0,Мария,79990004444,Велосипед,ЦАО
5,5,2025-03-17 14:00:00,Дмитрий,"ул. Садовая, 25",Курьерская доставка,700.0,Иван,79990005555,Автомобиль,ЮВАО
...,...,...,...,...,...,...,...,...,...,...
148,148,2025-03-19 07:00:00,Лидия,"ул. Звездная, 19",Доставка еды,500.0,Алексей,79990001111,Автомобиль,ЦАО
149,149,2025-03-19 08:00:00,Константин,"пер. 3-й, 25",Курьерская доставка,700.0,Дмитрий,79990002222,Велосипед,ЮВАО
150,150,2025-03-19 09:00:00,Александра,"пр. Свободы, 11",Экспресс-доставка,1200.0,АНтон,79990003333,Мотоцикл,ЗАО
151,151,2025-03-19 10:00:00,Геннадий,"пер. Тростниковый, 8",Доставка еды,500.0,Мария,79990004444,Велосипед,ЦАО


In [9]:
#CRUD
#делаем курьера Ваню
execute("""
    insert into
      Couriers (name, phone, transport_type, zone)
    values
      ('Ваня', '88888888888', 1, 3);
""")

sql('select * from  Couriers;')

,courier_id,name,phone,transport_type,zone
1,1,Алексей,79990001111,1,1
2,2,Дмитрий,79990002222,2,2
3,3,АНтон,79990003333,3,3
4,4,Мария,79990004444,2,1
5,5,Иван,79990005555,1,2
6,6,Алексей,79990001111,1,1
7,7,Дмитрий,79990002222,2,2
8,8,АНтон,79990003333,3,3
9,9,Мария,79990004444,2,1
10,10,Иван,79990005555,1,2


In [10]:
# Иван поднялся(или опустился) и теперь разьезжает на самокате
execute("""
          update
                Couriers
          SET
                transport_type = 4
          WHERE
                name = 'Иван';

""")

sql('select * from Couriers')

,courier_id,name,phone,transport_type,zone
1,1,Алексей,79990001111,1,1
2,2,Дмитрий,79990002222,2,2
3,3,АНтон,79990003333,3,3
4,4,Мария,79990004444,2,1
5,5,Иван,79990005555,4,2
6,6,Алексей,79990001111,1,1
7,7,Дмитрий,79990002222,2,2
8,8,АНтон,79990003333,3,3
9,9,Мария,79990004444,2,1
10,10,Иван,79990005555,4,2


In [11]:
#Ваня выбрал быть счастливым и уволился
execute("""
            delete from
                  Couriers
            where
                  phone = '88888888888'
""")
sql('select * from Couriers')

,courier_id,name,phone,transport_type,zone
1,1,Алексей,79990001111,1,1
2,2,Дмитрий,79990002222,2,2
3,3,АНтон,79990003333,3,3
4,4,Мария,79990004444,2,1
5,5,Иван,79990005555,4,2
6,6,Алексей,79990001111,1,1
7,7,Дмитрий,79990002222,2,2
8,8,АНтон,79990003333,3,3
9,9,Мария,79990004444,2,1
10,10,Иван,79990005555,4,2


In [12]:
# Where с объединением трёх таблиц в where
#     (без использования join!) (0.4 балла)
# а тут смотрим на заказы сделанные на улицу Ленина, стоимостью более 600(извразение еще то)

sql("""
        select
              *
        from
              Orders
        where
              customer_id IN (select customer_id
                              from Customers
                              where address REGEXP('.+Ленина.+'))
        AND
              service_id IN (SELECT service_id
                             FROM Services
                            WHERE price > 600)
        AND
              courier_id IN (SELECT courier_id
                             FROM Couriers
                             WHERE transport_type = 2);

""")

,order_id,customer_id,service_id,courier_id,order_date,delivery_starts,delivery_ends
1,6,1,3,2,2025-03-17 15:00:00,2025-03-17 15:10:00,2025-03-17 16:25:00


In [13]:
#смотрим кто у нас терроризирует жителей самокатми
# 3. Вложенный select (1 балл):
#     3.1. Какой-нибудь - 0.3
#     3.2. Который нельзя переписать без вложенного запроса - 0.5
#     3.3. с with - 0.2

sql("""
        select
                *
        from
                Couriers
        where
                transport_type = (select transport_id
                                  from Transport_Types
                                  where transport_name = 'СИМ');
""")

,courier_id,name,phone,transport_type,zone
1,5,Иван,79990005555,4,2
2,10,Иван,79990005555,4,2
3,11,Светлана,79990006666,4,4
4,12,Николай,79990007777,4,4
5,13,Роман,79990008888,4,3
6,15,Георгий,79990010000,4,1


In [14]:
#смотрим на самых нищих клиентов
# 3.2. Который нельзя переписать без вложенного запроса - 0.5
sql("""
        select
              *
        from
              Orders
        where
              service_id = (select service_id
                            from Services
                            where price = (select min(price)
                                          from Services));
""")

,order_id,customer_id,service_id,courier_id,order_date,delivery_starts,delivery_ends
1,4,4,1,4,2025-03-17 13:00:00,2025-03-17 13:10:00,2025-03-17 13:41:00
2,7,2,1,3,2025-03-17 16:00:00,2025-03-17 16:10:00,2025-03-17 16:50:00
3,10,5,1,1,2025-03-17 19:00:00,2025-03-17 19:10:00,2025-03-17 19:47:00
4,11,1,1,3,2025-03-17 08:15:00,2025-03-17 08:25:00,2025-03-17 09:21:00
5,14,4,1,4,2025-03-17 11:45:00,2025-03-17 11:55:00,2025-03-17 12:18:00
6,17,7,1,8,2025-03-17 15:30:00,2025-03-17 15:40:00,2025-03-17 16:13:00
7,20,10,1,10,2025-03-17 19:15:00,2025-03-17 19:25:00,2025-03-17 20:36:00
8,23,13,1,6,2025-03-18 09:00:00,2025-03-18 09:10:00,2025-03-18 10:03:00
9,26,16,1,2,2025-03-18 12:45:00,2025-03-18 12:55:00,2025-03-18 13:29:00
10,29,19,1,10,2025-03-18 16:30:00,2025-03-18 16:40:00,2025-03-18 17:25:00


In [15]:
#смотрим на ранних пташек
sql("""
        select
                *
        from
                Orders
        where time(order_date) = (select min(time(order_date))
                            from Orders
                            where time(order_date) > '04:00:00') ;
""")

,order_id,customer_id,service_id,courier_id,order_date,delivery_starts,delivery_ends
1,122,20,2,10,2025-03-18 05:00:00,2025-03-18 05:10:00,2025-03-18 05:45:00
2,146,44,2,4,2025-03-19 05:00:00,2025-03-19 05:10:00,2025-03-19 05:58:00


In [16]:
#смотрим скольким клиентам приспичило заказать до 10:30, и кто уже не спал из курьеров
sql("""
        with cte as (
              select
                      order_id
                    , o.courier_id
                    , start_time
                    , end_time
                    , order_date
              from
                      Orders o
              inner join
                      Courier_Schedule cs on cs.courier_id = o.courier_id)
          select
                  courier_id
                , count(order_id) as amount
          from
                  cte
          where
                  '10:30:00' > time(order_date)
          group by
                  courier_id;
""")

,courier_id,amount
1,1,4
2,2,2
3,3,3
4,4,3
5,5,7
6,6,4
7,7,5
8,8,3
9,9,5
10,10,3


In [17]:
#думаем расширяться и смотрим где у нас не хвататет курьеров(может перебросить из "хлебных" округов)
# Join - 0.5, left/right join к месту - 0.5 (итого - 1 балл)
# left/right join + group by + having

sql("""
        select
                z.zone_name
              , count(c.name) carrier_num
        from
                Zones z
        left join
                Couriers c ON c.zone = z.zone_id
        group by
                zone_id
        having
                carrier_num<5""")

,zone_name,carrier_num
1,ЗАО,3
2,САО,2
3,Химки,0


In [18]:
#смотрим кто позже всего жует
#order by
sql("""
        select
                *
        from
                Orders O
        inner join
                Services using(service_id)
        order by
                time(order_date) desc
        limit 1;""")

,service_id,order_id,customer_id,courier_id,order_date,delivery_starts,delivery_ends,service_name,price
1,1,95,35,8,2025-03-23 23:00:00,2025-03-23 23:10:00,2025-03-24 00:26:00,Доставка еды,500.0


In [19]:
# время грести капусту в день(в среднем)
# аггрегация в результирующих полях - 0.2
sql("""
        select
                count(order_id) total_orders
              , avg(price)  avg_price
        from
                Orders o
        inner join
                Services s on s.service_id = o.service_id
        where
                date(order_date) = '2025-03-23';
""")

,total_orders,avg_price
1,13,776.923077


In [20]:
# а вот и наши золотоносные куры
#аггрегация в where - 0.2
sql("""
          select
                count(order_id) total_orders, group_concat(distinct customer_id) as customers
          from
                Orders o
          inner join
                Services s on s.service_id = o.service_id
          where
                price>(select avg(price) from Orders o inner join Services s on s.service_id = o.service_id);""")

,total_orders,customers
1,49,"1,3,4,6,7,9,10,12,13,15,16,18,19,21,22,24,25,2..."


In [30]:
# пропишем нимного правил чтобы наш бизнес не ушел в тартарары
# 1) не создаем заказы в прошлом
# 2) ну и делаем упрощенную процеду создания заказов

#trigger
# DELIMITER //
# CREATE TRIGGER before_order_insert
# BEFORE INSERT ON Orders
# FOR EACH ROW
# BEGIN
#     IF NEW.order_date < NOW() THEN
#         SIGNAL SQLSTATE '45000'
#         SET MESSAGE_TEXT = 'Дата заказа не может быть в прошлом';
#     END IF;
# END;
# //
# DELIMITER ;

#procedure
# CREATE PROCEDURE create_order(
#     IN p_customer_id INT,
#     IN p_service_id INT,
#     IN p_courier_id INT,
#     IN p_order_date DATETIME,
#     IN p_delivery_starts DATETIME,
#     IN p_delivery_ends DATETIME
# )
# BEGIN

#     IF (SELECT COUNT(*) FROM Customers WHERE customer_id = p_customer_id) = 0 THEN
#         SIGNAL SQLSTATE '45000'
#         SET MESSAGE_TEXT = 'client error';
#     END IF;

#     IF (SELECT COUNT(*) FROM Couriers WHERE courier_id = p_courier_id) = 0 THEN
#         SIGNAL SQLSTATE '45000'
#         SET MESSAGE_TEXT = 'courier error';
#     END IF;

#     IF (SELECT COUNT(*) FROM Services WHERE service_id = p_service_id) = 0 THEN
#         SIGNAL SQLSTATE '45000'
#         SET MESSAGE_TEXT = 'service error';
#     END IF;

#
#     INSERT INTO Orders (
#         customer_id,
#         service_id,
#         courier_id,
#         order_date,
#         delivery_starts,
#         delivery_ends
#     )
#     VALUES (
#         p_customer_id,
#         p_service_id,
#         p_courier_id,
#         p_order_date,
#         p_delivery_starts,
#         p_delivery_ends
#     );

#     SELECT LAST_INSERT_ID() AS new_order_id;
# END;
# //
# DELIMITER ;

sql("""CALL create_order(1, 2, 3, '2022-03-20 10:00:00', '2025-03-20 11:00:00', '2025-03-20 12:00:00');""")
# sql("""CALL create_order(1, 2, 3, '2025-03-20 10:00:00', '2025-03-20 11:00:00', '2025-03-20 12:00:00');""")

OperationalError: (pymysql.err.OperationalError) (1644, 'Дата заказа не может быть в прошлом')
[SQL: CALL create_order(1, 2, 3, '2022-03-20 10:00:00', '2025-03-20 11:00:00', '2025-03-20 12:00:00');]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [31]:
# если вдруг чего-то напортачили, то можно подправить

# DELIMITER //

# CREATE PROCEDURE update_order(
#     IN p_order_id INT,
#     IN p_customer_id int,
#     in p_service_id int,
#     in courier_id int,
#     IN p_order_date DATETIME,
#     IN p_delivery_starts DATETIME,
#     IN p_delivery_ends DATETIME

# )
# BEGIN
#
#     IF (SELECT COUNT(*) FROM Orders WHERE order_id = p_order_id) = 0 THEN
#         SIGNAL SQLSTATE '45000'
#         SET MESSAGE_TEXT = 'Ошибка: заказ с таким ID не найден';
#     END IF;

#
#     UPDATE Orders
#     SET
#         order_date = p_order_date,
#         delivery_starts = p_delivery_starts,
#         delivery_ends = p_delivery_ends,
#         customer_id = p_customer_id,
#         service_id = p_service_id,
#         courier_id = p_courier_id
#     WHERE order_id = p_order_id;
# END;
# //
# DELIMITER ;


# CALL update_order(1, 3, 2, 4, '2025-03-19 15:00:00', '2025-03-19 15:30:00', '2025-03-19 16:00:00');
execute("""CALL update_order(1, 3, 2, 4, '2025-03-19 15:00:00', '2025-03-19 15:30:00', '2025-03-19 16:00:00')""")

In [23]:
# CREATE INDEX idx_order_date ON mysqlproject_2.Orders (order_date);
# SHOW INDEX FROM Orders;

In [32]:
# а вот тут пишем простенький скрипт для деланья бекапов нашей бд, чтобы ни клиенты, ни куртеры не растерлись

bucket_name = '***'
ac_key = "****"
sec_key = "***"
bucket_endpoint  = "hb.ru-msk.vkcloud-storage.ru"

def dump_to_s3(bucket_name, ac_key, sec_key, bucket_endpoint):
  """
    Функция для отправки состояния всех таблиц из базы данных в S3 хранилище.

    Аргументы:
    bucket_name (str): Название S3 бакета, куда будет загружен архив.
    ac_key (str): Access key для доступа к S3.
    sec_key (str): Secret key для доступа к S3.
    bucket_endpoint (str): Доменный адрес конечной точки S3.

    Описание:
    Функция создает архив, содержащий CSV-файлы всех таблиц из базы данных, и загружает этот архив в указанный S3 бакет.
    Таблицы выгружаются из базы данных в формате CSV. После этого создается ZIP-архив, который загружается в S3.
    """

  cur_time=datetime.utcnow() + timedelta(hours=3)
  dir_name = cur_time.strftime("%Y%m%d_%H%M")
  os.makedirs('to_dump', exist_ok=True)


  for table in (sql('show tables;').values.flatten().tolist()):
      df = sql(f'select * from {table};')
      csv_file = f"to_dump/{table}.csv"
      df.to_csv(csv_file, index=False, encoding="utf-8")


  zip_file = f"{dir_name}.zip"
  os.system(f"zip -r {zip_file} to_dump")


  s3 = boto3.client(
    "s3",
    endpoint_url="https://hb.vkcloud-storage.ru",
    aws_access_key_id=ac_key,
    aws_secret_access_key=sec_key,
    config=boto3.session.Config(region_name='ru-msk', signature_version="s3")
  )

  s3_key = f"sqlbackups/{zip_file}"

  with open(f"{zip_file}", "rb") as file_obj:
    s3.upload_fileobj(file_obj, bucket_name, s3_key, ExtraArgs={'ContentType': 'application/zip'})

  print(f'УСПЕХ!, ищи файл тут:s3://{bucket_name}/{s3_key}')

dump_to_s3(bucket_name, ac_key, sec_key, bucket_endpoint)

#https://msk.cloud.vk.com/app/mcs3072380851/services/storage/buckets/retail-perfilev?Prefix=sqlbackups%2F

УСПЕХ!, ищи файл тут:s3://retail-perfilev/sqlbackups/20250321_1518.zip


In [35]:
sql('call full_info(3)').head()

,courier_id,order_id,order_date,delivery_starts,delivery_ends,name,address,service_name,price,name,phone,transport_name,zone_name
1,3,7,2025-03-17 16:00:00,2025-03-17 16:10:00,2025-03-17 16:50:00,Ольга,"ул. Гагарина, 10",Доставка еды,500.0,АНтон,79990003333,Мотоцикл,ЗАО
2,3,11,2025-03-17 08:15:00,2025-03-17 08:25:00,2025-03-17 09:21:00,Павел,"ул. Ленина, 5",Доставка еды,500.0,АНтон,79990003333,Мотоцикл,ЗАО
3,3,50,2025-03-20 11:45:00,2025-03-20 11:55:00,2025-03-20 12:38:00,Виктор,"ул. Светланова, 39",Доставка еды,500.0,АНтон,79990003333,Мотоцикл,ЗАО
4,3,86,2025-03-23 11:45:00,2025-03-23 11:55:00,2025-03-23 12:17:00,Николай,"ул. Тверская, 50",Доставка еды,500.0,АНтон,79990003333,Мотоцикл,ЗАО
5,3,115,2025-03-17 22:00:00,2025-03-17 22:10:00,2025-03-17 23:21:00,Михаил,"ул. Речная, 65",Доставка еды,500.0,АНтон,79990003333,Мотоцикл,ЗАО


In [36]:
sql('''
      with cte as(
          select
                  distinct c.name
                , date(order_date) date
                , sum(price) over w as sum_per_date
                , count(order_id) over w as orders_per_date
          from
                  Orders
          inner join
                  Couriers c using(courier_id)
          inner join
                  Services s using(service_id)
          window
                  w as(partition by
                            courier_id,
                            date(order_date)
                        order by
                            date(order_date))
          order by
                  date(order_date)
                , sum_per_date desc
                , orders_per_date desc)


          select
                  *
                , dense_rank()
                    over (partition by
                              date
                          order by
                              sum_per_date desc) rank_per_sum
                , dense_rank()
                    over (partition by
                              date
                          order by
                              orders_per_date desc) rank_per_orders
          from
                  cte
          order by
                  date
                , rank_per_sum
                , rank_per_orders
          ''')


,name,date,sum_per_date,orders_per_date,rank_per_sum,rank_per_orders
1,Дмитрий,2025-03-17,5000.0,5,1,2
2,АНтон,2025-03-17,4600.0,6,2,1
3,Иван,2025-03-17,4500.0,5,3,2
4,Мария,2025-03-17,2900.0,5,4,2
5,Алексей,2025-03-17,2400.0,4,5,3
...,...,...,...,...,...,...
67,Мария,2025-03-24,700.0,1,2,1
68,Дмитрий,2025-03-24,700.0,1,2,1
69,Алексей,2025-03-24,700.0,1,2,1
70,Дмитрий,2025-03-24,500.0,1,3,1


In [27]:
sql('''
with cte as(
select distinct c.name, courier_id, date(order_date) date, sum(price) over w as sum_per_date, count(order_id) over w as orders_per_date
from Orders
inner join Couriers c using(courier_id)
inner join Services s using(service_id)
window w as(partition by courier_id, date(order_date) order by date(order_date))
order by date(order_date), sum_per_date desc, orders_per_date desc),
cte2 as(
select *, dense_rank() over (partition by date order by sum_per_date desc) rank_per_sum, dense_rank() over (partition by date order by orders_per_date desc) rank_per_orders
from cte
order by date, rank_per_sum, rank_per_orders)

select name, courier_id, sum(rank_per_sum)+sum(rank_per_orders) as best_of_the_best
from cte2
group by courier_id
order by best_of_the_best;
''')

,name,courier_id,best_of_the_best
1,Мария,4,32.0
2,Иван,5,35.0
3,Дмитрий,2,38.0
4,АНтон,3,39.0
5,Дмитрий,7,39.0
6,Алексей,1,40.0
7,Алексей,6,41.0
8,АНтон,8,42.0
9,Мария,9,46.0
10,Иван,10,53.0
